# Preparing an AF state in the Ising model

This purpose of this notebook is to illustrate how to build a sequence for preparing an AF state in the Ising model. This notebook is supposed to evolve with code updates. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pulser import Pulse, Sequence, Register
from pulser.waveforms import ConstantWaveform, RampWaveform
from pulser.devices import AFM
from pulser.simulation import Simulation

simulation module...


## Initializing my register

In [ ]:
line_qubits = Register.square_1D(n_atoms = 8, spacing=6, prefix='q')
line_qubits.draw()

## Defining my waveforms 

We are realizing the following program

<img src="files/AF_Ising_program.png" alt="AF Pulse Sequence" style="width: 320px;"/>

In [ ]:
# Parameters in MHz and ns
U = 1 * 2*np.pi  # btw 1-3 Mhz*2pi. Should we include this VdW magnitude in an AFM-type Simulation?

delta_0 = -2 * 2*np.pi
delta_f = 2 * 2*np.pi
Omega_max = 1. * 2*np.pi  # btw 1.8-2 Mhz*2pi
t_rise = 250
t_fall = 250
t_sweep = (delta_f-delta_0)/(2*np.pi*10) * 1000


In [ ]:
## Creating my sequence

In [ ]:
af_sequence = Sequence(my_device)
af_sequence.declare_channel('Ising_channel', 'rydberg_global')

af_sequence.add(rise, 'Ising_channel')
af_sequence.add(sweep, 'Ising_channel')
af_sequence.add(fall, 'Ising_channel')

print(af_sequence)
af_sequence.draw()

In [ ]:
sweep.amplitude.integral

2.5132741228718345

# Simulation

In [ ]:
af_sequence = Sequence(line_qubits, Chadoq2)

In [ ]:
import qutip

up = qutip.basis(2,0)
down = qutip.basis(2,1)

all_down = qutip.tensor([down for _ in range(simul._size)])

In [ ]:
def magnetization(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = qutip.sigmaz()
    return qutip.tensor(prod)

def occupation(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)
    
magne_list = [magnetization(j) for j in range(simul._size)]
occup_list = [occupation(j) for j in range(simul._size//2)]

In [ ]:
By parity symmetry, we only need to observe half of the atoms, the other one reflects the results

In [ ]:
simul.run(obs_list=magne_list)

In [ ]:
L = 3
res = np.zeros((2,3))
for i, m in enumerate(simul.output):
    res[i%2,i//3] = m[-1]
    
plt.matshow(res)